# Entrenamiento y evaluación de modelo experimento 3
* **Autor:** Julian Zuloaga
* **Asignatura:** Memoria de Título
* **Fecha:** Agosto de 2022
---

**Descripción: [Notas base de datos]**
Para entrenar este modelo (YOLOv4_Mosaic_Dataset_v2), la base de datos fue elaborada combinando los Custom Mosaics generados considerando 9, 80 y 200, rostros de AffectNet por imagen de mosaico.



# Paso 1: Copia de repositorio

In [ ]:
# Se clona el repositorio de Alexey Bochkovskiy con YOLOv4 y el framework Darknet
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Se modifica el archivo makefile para habilitar el uso de GPU y OPENCV
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# Se verifica versión de CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# Se descarga archivo detector.c modificado para habilitar respaldos de pesos cada 1000 iteraciones de entrenamiento
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=125EdgI2LgII3ihVpcUEN96tz8L37t2Zw' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=125EdgI2LgII3ihVpcUEN96tz8L37t2Zw" -O /content/darknet/src/detector.c && rm -rf /tmp/cookies.txt

In [ ]:
# make darknet
!make

# Paso 2: Funciones auxiliares

In [ ]:
# Función para mostrar imágenes con cv2
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# Función para descargar archivos desde Colab a nuestro PC local  
def download(path):
  from google.colab import files
  files.download(path)

# Paso 3: Se monta unidad de Google Drive


In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Se crea un symbolic link para reemplazar "/content/gdrive/My\ Drive/" a "/mydrive"
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# Se vuelve a la carpeta principal de darknet
%cd darknet

# Paso 4: Carga de Base de datos

In [ ]:
## Conjuntos generados con counting = 9
# Se copian los archivos comprimidos de la base de datos
!cp /mydrive/YOLOv4_Mosaic_Dataset_v2/Mosaic_Dataset_v9/C9/obj.zip ../
!cp /mydrive/YOLOv4_Mosaic_Dataset_v2/Mosaic_Dataset_v9/C9/test.zip ../
# se descomprimen los archivos con los conjuntos de entrenamiento y test en /darknet/data/
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/
# Se eliminan los comprimidos que ya no se usarán
!rm ../obj.zip ../test.zip

In [ ]:
## Conjuntos generados con counting = 80
# Se copian los archivos comprimidos de la base de datos
!cp /mydrive/YOLOv4_Mosaic_Dataset_v2/Mosaic_Dataset_v9/C80/obj.zip ../
!cp /mydrive/YOLOv4_Mosaic_Dataset_v2/Mosaic_Dataset_v9/C80/test.zip ../
# se descomprimen los archivos con los conjuntos de entrenamiento y test en /darknet/data/
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/
# Se eliminan los comprimidos que ya no se usarán
!rm ../obj.zip ../test.zip

In [ ]:
## Conjuntos generados con counting = 200
# Se copian los archivos comprimidos de la base de datos
!cp /mydrive/YOLOv4_Mosaic_Dataset_v2/Mosaic_Dataset_v9/C200/obj.zip ../
!cp /mydrive/YOLOv4_Mosaic_Dataset_v2/Mosaic_Dataset_v9/C200/test.zip ../
# se descomprimen los archivos con los conjuntos de entrenamiento y test en /darknet/data/
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/
# Se eliminan los comprimidos que ya no se usarán
!rm ../obj.zip ../test.zip

# Paso 5: Carga de archivos de configuración

In [ ]:
# Se carga archivo de configuración .cfg desde Google Drive
!cp /mydrive/YOLOv4_Mosaic_Dataset_v1_Graficos/yolov4-obj.cfg ./cfg

In [ ]:
# Se suben los archivos obj.names y obj.data desde Google Drive
!cp /mydrive/YOLOv4_Mosaic_Dataset_v2/obj.names ./data
!cp /mydrive/YOLOv4_Mosaic_Dataset_v2/obj.data  ./data

In [ ]:
# Se suben los archivos generate_train.py y generate_test.py desde Google Drive
!cp /mydrive/YOLOv4_Mosaic_Dataset_v2/generate_train.py ./
!cp /mydrive/YOLOv4_Mosaic_Dataset_v2/generate_test.py ./

In [ ]:
# Se ejecutan los archivos recien subidos
!python generate_train.py
!python generate_test.py

In [ ]:
# Se verifica que se haya generado los archivos train.txt test.txt
!ls data/

In [ ]:
# Se lanza error para detener ejecución de todas las celdas
assert False

# Paso 6: Carga de pesos pre entrenados

In [ ]:
# Estos pesos se descargan del mismo repositorio que Darknet
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# Paso 7: Inicio de entrenamiento

In [ ]:
# Se comienza el entrenamiento de la red! (descomentar %%capture si ocurren problemas de memoria o Colab lanza algún error)
# %%capture
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

# Paso 8: Reanudación de entrenamiento

In [ ]:
# kick off training from where it last saved
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/YOLOv4_Mosaic_Dataset_v2/backup/yolov4-obj_6000.weights -dont_show

# Paso 9: Revisión de la precisión

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/YOLOv4_Mosaic_Dataset_v2/backup/yolov4-obj_6000.weights

# Paso 10: Pruebas de detección

In [ ]:
# Se modifica el archivo de configuración .cfg para correr pruebas
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

In [ ]:
# Se ejecuta detección
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/YOLOv4_Mosaic_Dataset_v2/backup/yolov4-obj_6000.weights /mydrive/YOLOv4_Mosaic_Dataset_v2/IMG_22_masked.jpg -thresh 0.3
imShow('predictions.jpg')

In [ ]:
# Se vuelve a modificar archivo .cfg para que se pueda reanudar entrenamiento
%cd cfg
!sed -i 's/batch=1/batch=64/' yolov4-obj.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' yolov4-obj.cfg
%cd ..

Fin del programa